In [1]:
#Dataset Link : https://www.kaggle.com/competitions/titanic/data

In [2]:
import numpy as np
import pandas as pd        # For loading and processing the dataset
import tensorflow as tf    # Of course, we need TensorFlow.
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

In [3]:
# Read the CSV input file and show first 5 rows
df_train = pd.read_csv('../datasets/train.csv')
df_train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# We can't do anything with the Name, Ticket number, and Cabin, so we drop them.
df_train = df_train.drop(['PassengerId','Name','Ticket', 'Cabin'], axis=1)

In [5]:
# To make 'Sex' numeric, we replace 'female' by 0 and 'male' by 1
df_train['Sex'] = df_train['Sex'].map({'female':0, 'male':1}).astype(int) 

In [6]:
# We replace 'Embarked' by three dummy variables 'Embarked_S', 'Embarked_C', and 'Embarked Q',
# which are 1 if the person embarked there, and 0 otherwise.
df_train = pd.concat([df_train, pd.get_dummies(df_train['Embarked'], prefix='Embarked')], axis=1)
df_train = df_train.drop('Embarked', axis=1)

In [7]:
# We normalize the age and the fare by subtracting their mean and dividing by the standard deviation
age_mean = df_train['Age'].mean()
age_std = df_train['Age'].std()
df_train['Age'] = (df_train['Age'] - age_mean) / age_std

fare_mean = df_train['Fare'].mean()
fare_std = df_train['Fare'].std()
df_train['Fare'] = (df_train['Fare'] - fare_mean) / fare_std

In [8]:
# In many cases, the 'Age' is missing - which can cause problems. Let's look how bad it is:
print("Number of missing 'Age' values: {:d}".format(df_train['Age'].isnull().sum()))

# A simple method to handle these missing values is to replace them by the mean age.
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].mean())

Number of missing 'Age' values: 177


In [9]:
# With that, we're almost ready for training
df_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,-0.530005,1,0,-0.502163,0,0,1
1,1,1,0,0.571430,1,0,0.786404,1,0,0
2,1,3,0,-0.254646,0,0,-0.488580,0,0,1
3,1,1,0,0.364911,1,0,0.420494,0,0,1
4,0,3,1,0.364911,0,0,-0.486064,0,0,1


In [10]:
# Finally, we convert the Pandas dataframe to a NumPy array, and split it into a training and test set
X_train = df_train.drop('Survived', axis=1).values
y_train = df_train['Survived'].values

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2)

In [11]:
# Define the model
model = models.Sequential()

input_dim = 9
output_dim = 2
# Add input layer
model.add(layers.Input(shape=(input_dim,)))

# Add hidden layers
model.add(layers.Dense(units=64, activation='relu'))
model.add(layers.Dense(units=32, activation='relu'))

# Add output layer
model.add(layers.Dense(units=output_dim, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                640       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 2)                 66        
                                                                 
Total params: 2,786
Trainable params: 2,786
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Training the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Evaluating the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

Epoch 1/10
23/23 [==============================] - 2s 3ms/step - loss: 0.6869 - accuracy: 0.5478
Epoch 2/10
23/23 [==============================] - 0s 3ms/step - loss: 0.5918 - accuracy: 0.6756
Epoch 3/10
23/23 [==============================] - 0s 3ms/step - loss: 0.5526 - accuracy: 0.7219
Epoch 4/10
23/23 [==============================] - 0s 4ms/step - loss: 0.5223 - accuracy: 0.7430
Epoch 5/10
23/23 [==============================] - 0s 3ms/step - loss: 0.4952 - accuracy: 0.7739
Epoch 6/10
23/23 [==============================] - 0s 3ms/step - loss: 0.4724 - accuracy: 0.7978
Epoch 7/10
23/23 [==============================] - 0s 3ms/step - loss: 0.4575 - accuracy: 0.8090
Epoch 8/10
23/23 [==============================] - 0s 4ms/step - loss: 0.4429 - accuracy: 0.8188
Epoch 9/10
23/23 [==============================] - 0s 4ms/step - loss: 0.4358 - accuracy: 0.8202
Epoch 10/10
6/6 [==============================] - 0s 3ms/step - loss: 0.4550 - accuracy: 0.7989
Test accuracy: 0.7988